In [1]:
import numpy as np
import pandas as pd
import copy as cp

In [2]:
def show_df(df,row=10):
    '''
        dis play the first few line of a dataframe
        '''
    print(df[0:row])
def p_init(training_set):
    '''
        get prior prob in the dataframe
        '''
    movie_list=training_set.mid.drop_duplicates().sort_values().values
    p_list=[]
    #print(movie_list)
    for movie in movie_list:
        rat_all=training_set[training_set.mid==movie]
        rat_good=rat_all[rat_all.rating==1]
        p=len(rat_good)/len(rat_all)
        p_list.append(p)
    #print(p_list)
    return p_list

In [3]:
class AdamOpti:
    def __init__(self,array1,delta,lr,beta1,beta2,epi):
        self.m=delta
        self.v=np.power(delta,2)
        self.lr=lr
        self.beta1=beta1
        self.beta2=beta2
        self.epi=epi
        self.t=0
    def opt(self,array,delta):
        self.t+=1
        try:
            self.m=self.m*(self.beta1)+delta*(1-self.beta1)
        except ValueError:
            print(self.m.shape)
            print(delta.shape)
        self.v=self.v*(self.beta2)+np.power(delta,2)*(1-self.beta2)
        mhat=self.m/(1-self.beta1**self.t)
        vhat=self.v/(1-self.beta2**self.t)
        return array+self.lr*(mhat/(self.epi+np.power(vhat,0.5)))
                
#read table
j=0
hidden=30
col=['uid','mid','rating','useless']
rating=pd.read_table('ratings.dat',sep = '::',names=col,engine='python')
rating=rating.drop(['useless'],axis=1)
rating=rating[rating.mid<4001]
#split rating to good or bad

In [4]:
rating_keep=cp.deepcopy(rating)

In [5]:
ab=rating.rating
ab[ab<6]=1
rating.rating=ab
print(len(rating))
#split to train and test

all_col=rating[rating.uid>=2001].pivot(index='uid',columns='mid',values='rating').columns\
         .intersection\
         (rating[rating.uid<2001].pivot(index='uid',columns='mid',values='rating').columns)
criterion = rating['mid'].map(lambda x: x in all_col.values)
rating=rating[criterion]
training_set=rating[rating.uid<2001]
test_set=rating[rating.uid>=2001]
#get uid-mid matrix
train_mat=training_set.pivot(index='uid',columns='mid',values='rating').fillna(value=0).values
test_mat=rating[rating.uid>=2001].pivot(index='uid',columns='mid',values='rating').fillna(value=0).values
visible=test_mat.shape[1]

1000209


In [6]:
def df_count(df,values=0):
    a=np.array([[1,2,3],[4,5,6],[1,1,1],[4,4,6]])
    a=pd.DataFrame(a)
    if type(a)==type(df):
        return np.sum((df==values).astype(int).values)
    return np.sum((pd.DataFrame(df)==values).astype(int).values)

In [8]:
class RBM_see:
    def __init__(self,hidden,visible,no=0):
        self.hidd_bias=np.zeros((hidden,1))
        self.visi_bias=np.zeros((visible,1))
        self.weight=np.zeros((hidden,visible))
        self.hidden=hidden
        self.visi=visible
        self.no=no
        self.opt=None
        self.opt1=None
        self.opt3=None
        self.opt2=None
        #print(self.visi_bias.shape)
    def save(self):
        hidd=pd.DataFrame(self.hidd_bias)
        visi=pd.DataFrame(self.visi_bias)
        weight=pd.DataFrame(self.weight)
        #print(hidd)
        hidd.to_csv('hidd_'+str(self.no)+'.csv',index=False)
        visi.to_csv('visi_'+str(self.no)+'.csv',index=False)
        weight.to_csv('weight_'+str(self.no)+'.csv',index=False)
    def load(self):
        try:
            self.hidd_bias=pd.read_csv('hidd_'+str(self.no)+'.csv').values
            #print(pd.read_csv('hidd.csv',index_col=False))
            self.visi_bias=pd.read_csv('visi_'+str(self.no)+'.csv').values
            self.weight=pd.read_csv('weight_'+str(selfno)+'.csv').values
        except Exception:
            return 0
    def predict(self,test_mat,use_mov=2000):
        n=test_mat.shape[0]
        fake_test=cp.deepcopy(test_mat)
        fake_test[:,use_mov:]=0
        hidden_infer_prob=np.dot(fake_test,self.weight.T)+np.repeat(self.hidd_bias.T,n,axis=0)
        hidden_infer_prob=1/(1+np.exp(-hidden_infer_prob))
        #print(self.visi_bias.shape)
        hidden_infer=np.random.binomial(1,hidden_infer_prob)
        viss_infer_prob=np.dot(hidden_infer,self.weight)+np.repeat(self.visi_bias.T,n,axis=0)
        viss_infer_prob=1/(1+np.exp(-viss_infer_prob))
        viss_infer=np.random.binomial(1,viss_infer_prob)
        return viss_infer
    def test(self,test_mat,use_mov=2000):
        viss_infer=self.predict(test_mat,use_mov=use_mov)
        n=test_mat.shape[0]
        p=test_mat.shape[1]
        test_mat[:,use_mov:]=0
        viss_infer[:,use_mov:]=0
        return np.sum(np.power(viss_infer-test_mat,2))/(n*(p-use_mov))
    def detail_test(self,test_mat,use_mov=2000):
        viss_infer=self.predict(test_mat,use_mov=use_mov)
        n=test_mat.shape[0]
        p=test_mat.shape[1]
        test_mat[:,use_mov:]=2
        viss_infer[:,use_mov:]=2
        zero=np.where(test_mat==0,viss_infer,5)
        one=np.where(test_mat==1,viss_infer,5)
        print('t:1,predict:1--'+str(df_count(one,1)))
        print('t:1,predict:0--'+str(df_count(one,0)))
        print('t:0,predict:1--'+str(df_count(zero,1)))
        print('t:0,predict:0--'+str(df_count(zero,0)))
        
    

        
        
    def train(self,train_mat,lr=0.001,show_error=True,train_round=1000,
              mode='gd',beta1=0.9,beta2=0.999,epi=0.00000001,show_term=10):
        n=train_mat.shape[0]
        train_init=np.where(train_mat==2,0,train_mat)
        #print(self.visi_bias.shape)
        train_mat=train_init
        for i in range(train_round):
            error=0
            hidden_infer_prob=np.dot(train_mat,self.weight.T)+np.repeat(self.hidd_bias.T,n,axis=0)
            hidden_infer_prob=1/(1+np.exp(-hidden_infer_prob))
            #print(self.visi_bias.shape)
            hidden_infer=np.random.binomial(1,hidden_infer_prob)
            try:
                viss_infer_prob=np.dot(hidden_infer,self.weight)+np.repeat(self.visi_bias.T,n,axis=0)
            except ValueError:
                print(hidden_infer.shape)
                print(self.weight.shape)
                print(self.visi_bias.shape)
            viss_infer_prob=1/(1+np.exp(-viss_infer_prob))
            viss_infer=np.random.binomial(1,viss_infer_prob)
            '''
            train_infer=np.where(train_mat==2,viss_infer,train_mat)
            hidden_infer_prob=np.dot(train_infer,self.weight.T)+np.repeat(self.hidd_bias.T,n,axis=0)
            hidden_infer_prob=1/(1+np.exp(-hidden_infer_prob))
            #print(self.visi_bias.shape)
            hidden_infer=np.random.binomial(1,hidden_infer_prob)
            try:
                viss_infer_prob=np.dot(hidden_infer,self.weight)+np.repeat(self.visi_bias.T,n,axis=0)
            except ValueError:
                print(hidden_infer.shape)
                print(self.weight.shape)
                print(self.visi_bias.shape)
            viss_infer_prob=1/(1+np.exp(-viss_infer_prob))
            viss_infer=np.random.binomial(1,viss_infer_prob)'''
            hidden_1_infer_prob=np.dot(viss_infer,self.weight.T)+np.repeat(self.hidd_bias.T,n,axis=0)
            hidden_1_infer_prob=1/(1+np.exp(-hidden_1_infer_prob))
            hidden_1_infer=np.random.binomial(1,hidden_1_infer_prob)
            try:
                delta_weight=(1/n)*(np.dot(hidden_infer.T,train_mat)-np.dot(hidden_infer.T,viss_infer))
            except TypeError:
                print((hidden_infer.shape))
                print((train_mat.shape))
                print((viss_infer.shape))
    
            delta_viss=np.sum(train_mat-viss_infer,axis=0)*(1/n)
            #print(delta_viss.shape)
            delta_hidden=np.sum(hidden_infer-hidden_1_infer,axis=0)*(1/n)
            delta_hidden=delta_hidden.reshape(self.hidd_bias.shape)
            delta_viss=delta_viss.reshape(self.visi_bias.shape)
            error=np.sum(np.power(delta_weight,2))+np.sum(np.power(delta_viss,2))+np.sum(np.power(delta_hidden,2))
            if show_error==True:
                print(i)
                print(error)
            elif show_error=='special':
                if (i==0 or (((i+1)%show_term)==0)):
                    print(i)
                    print(error)
            if mode=='gd':
                self.weight=self.weight+delta_weight*lr
                self.visi_bias=self.visi_bias+delta_viss*lr
                self.hidd_bias=self.hidd_bias+delta_hidden*lr
            if mode=='adam':
                if self.opt==None:
                    self.opt1=AdamOpti(self.weight,delta_weight,lr,beta1,beta2,epi)
                    self.opt2=AdamOpti(self.visi_bias,delta_viss,lr,beta1,beta2,epi)
                    self.opt3=AdamOpti(self.hidd_bias,delta_hidden,lr,beta1,beta2,epi)
                    self.opt='gg'
                self.weight=self.opt1.opt(self.weight,delta_weight)
                self.visi_bias=self.opt2.opt(self.visi_bias,delta_viss)
                #print(self.visi_bias.shape)
                self.hidd_bias=self.opt3.opt(self.hidd_bias,delta_hidden)
                #print(self.hidd_bias.shape)

In [ ]:
hidden=50
rbm=RBM_see(hidden,visible,no=3)
rbm.load()
corr=rbm.detail_test(test_mat,use_mov=2000)
#print(corr)
while True:
    #print('c')
    rbm.train(train_mat,train_round=50,mode='adam',show_error='special')
    #print('d')
    rbm.save()
    corr=rbm.detail_test(test_mat,use_mov=2000)
    #print(corr)

t:1,predict:1--147576
t:1,predict:0--267137
t:0,predict:1--2587520
t:0,predict:0--5077767
0
6322.367902250001
9
4884.0416837500015
19
3985.538740000001
29
3239.0181745
39
2661.113682
49
2196.762031500001
t:1,predict:1--135987
t:1,predict:0--278726
t:0,predict:1--2281606
t:0,predict:0--5383681
0
2159.1710725000003
9
1855.2337272500001
19
1561.003562
29
1353.5744657500004
39
1173.78008875
49
1032.99371525
t:1,predict:1--133846
t:1,predict:0--280867
t:0,predict:1--2135940
t:0,predict:0--5529347
0
1026.3100525000002
9
915.87801075
19
828.0509840000002
29
748.7968695000001
39
693.9905667500001
49
629.74942425
t:1,predict:1--133559
t:1,predict:0--281154
t:0,predict:1--2055068
t:0,predict:0--5610219
0
628.0774747500001
9
587.4373469999999
19
544.9829375
29
510.87124675
39
479.0919902500001
49
457.8680995
t:1,predict:1--133098
t:1,predict:0--281615
t:0,predict:1--1994934
t:0,predict:0--5670353
0
455.6535370000001
9
434.4176065000001
19
413.83672250000006
29
396.99495525
39
378.3667775
49
363.0

9
38.7132695
19
38.69055725
29
38.1437255
39
37.729281
49
37.311997000000005
t:1,predict:1--85028
t:1,predict:0--329685
t:0,predict:1--938821
t:0,predict:0--6726466
0
37.3197685
9
36.88913875
19
36.415161250000004
29
36.11014925
39
35.644617749999995
49
35.40716125000001
t:1,predict:1--84143
t:1,predict:0--330570
t:0,predict:1--925819
t:0,predict:0--6739468
0
34.98778575000001
9
35.11904275
19
34.833056500000005
29
34.358280750000006
39
33.874075
49
33.32744600000001
t:1,predict:1--83196
t:1,predict:0--331517
t:0,predict:1--906871
t:0,predict:0--6758416
0
33.397696
9
33.216182249999996
19
32.71633825
29
32.4834065
39
31.962279250000005
49
31.651314000000003
t:1,predict:1--82336
t:1,predict:0--332377
t:0,predict:1--889047
t:0,predict:0--6776240
0
31.877272249999997
9
31.348574250000006
19
31.099084
29
30.55217225
39
30.35641225
49
29.9486755
t:1,predict:1--81761
t:1,predict:0--332952
t:0,predict:1--871049
t:0,predict:0--6794238
0
30.1491925
9
29.89400925
19
29.35539775
29
28.85407025
39

t:0,predict:1--491718
t:0,predict:0--7173569
0
3.67505375
9
3.6490099999999996
19
3.58062125
29
3.5418114999999997
39
3.51655575
49
3.4861240000000007
t:1,predict:1--69214
t:1,predict:0--345499
t:0,predict:1--485511
t:0,predict:0--7179776
0
3.48168875
9
3.44884475
19
3.36882025
29
3.3400505000000003
39
3.2806715
49
3.2902465000000003
t:1,predict:1--69375
t:1,predict:0--345338
t:0,predict:1--479057
t:0,predict:0--7186230
0
3.2756807500000003
9
3.27801875
19
3.1925457500000003
29
3.16574225
39
3.1282017500000006
49
3.09968
t:1,predict:1--68896
t:1,predict:0--345817
t:0,predict:1--474982
t:0,predict:0--7190305
0
3.0728677500000003
9
3.0785505
19
3.0384305000000005
29
3.0322457500000004
39
2.9789222499999997
49
2.9544460000000003
t:1,predict:1--69257
t:1,predict:0--345456
t:0,predict:1--469913
t:0,predict:0--7195374
0
2.9602805
9
2.9062060000000005
19
2.8684885
29
2.87498725
39
2.8312030000000004
49
2.8007487500000003
t:1,predict:1--68849
t:1,predict:0--345864
t:0,predict:1--465167
t:0,pre

t:0,predict:1--363338
t:0,predict:0--7301949
0
0.43541
9
0.434457
19
0.43662650000000003
29
0.42814775
39
0.43236025000000006
49
0.42875175000000004
t:1,predict:1--73700
t:1,predict:0--341013
t:0,predict:1--362411
t:0,predict:0--7302876
0
0.42310450000000005
9
0.42126275
19
0.42451825
29
0.41007475000000004
39
0.408691
49
0.419366
t:1,predict:1--74227
t:1,predict:0--340486
t:0,predict:1--362036
t:0,predict:0--7303251
0
0.411354
9
0.409374
19
0.41169075000000005
29
0.40472625
39
0.40395075
49
0.39327350000000005
t:1,predict:1--74478
t:1,predict:0--340235
t:0,predict:1--360809
t:0,predict:0--7304478
0
0.3896017500000001
9
0.39272975000000004
19
0.39166175000000003
29
0.3834962500000001
39
0.382344
49
0.38399300000000003
t:1,predict:1--74465
t:1,predict:0--340248
t:0,predict:1--359829
t:0,predict:0--7305458
0
0.37878300000000004
9
0.37235725000000003
19
0.36681325
29
0.37468674999999996
39
0.37243725000000005
49
0.3641515
t:1,predict:1--74737
t:1,predict:0--339976
t:0,predict:1--358828
t:

9
0.16417674999999998
19
0.16214325000000002
29
0.16252275
39
0.1603
49
0.1611605
t:1,predict:1--81300
t:1,predict:0--333413
t:0,predict:1--334598
t:0,predict:0--7330689
0
0.163821
9
0.16595975
19
0.15917
29
0.156802
39
0.16619849999999997
49
0.16288225
t:1,predict:1--81775
t:1,predict:0--332938
t:0,predict:1--336956
t:0,predict:0--7328331
0
0.15837800000000005
9
0.15784800000000002
19
0.15743275
29
0.15720725
39
0.15381499999999998
49
0.158926
t:1,predict:1--82322
t:1,predict:0--332391
t:0,predict:1--336409
t:0,predict:0--7328878
0
0.15964750000000003
9
0.16329849999999999
19
0.15976900000000002
29
0.159402
39
0.15591475000000002
49
0.15346225000000002
t:1,predict:1--82190
t:1,predict:0--332523
t:0,predict:1--334291
t:0,predict:0--7330996
0
0.15276825
9
0.159191
19
0.15696349999999998
29
0.16045125000000002
39
0.15336025
49
0.161214
t:1,predict:1--82276
t:1,predict:0--332437
t:0,predict:1--335036
t:0,predict:0--7330251
0
0.15435825
9
0.15754775000000001
19
0.1626545
29
0.1559972500000

...

t:0,predict:1--490890
t:0,predict:0--7174397
0
0.08094449999999999
9
0.07949175
19
0.0768935
29
0.08460625000000001
39
0.07958675
49
0.08083600000000002
t:1,predict:1--97827
t:1,predict:0--316886
t:0,predict:1--492596
t:0,predict:0--7172691
0
0.08567375
9
0.08261075000000001
19
0.08172600000000001
29
0.08155250000000001
39
0.08093999999999998
49
0.081084
t:1,predict:1--97807
t:1,predict:0--316906
t:0,predict:1--491555
t:0,predict:0--7173732
0
0.088038
9
0.08263975
19
0.0856055
29
0.07814575
39
0.08238950000000002
49
0.07790899999999999
t:1,predict:1--98037
t:1,predict:0--316676
t:0,predict:1--492216
t:0,predict:0--7173071
0
0.08359450000000002
9
0.0837235
19
0.08516225000000001
29
0.084941
39
0.08225450000000001
49
0.08310625
t:1,predict:1--98246
t:1,predict:0--316467
t:0,predict:1--491763
t:0,predict:0--7173524
0
0.08410625
9
0.08542775
19
0.082097
29
0.083554
39
0.084114
49
0.0832705
t:1,predict:1--97710
t:1,predict:0--317003
t:0,predict:1--490557
t:0,predict:0--7174730
0
0.084272
9


0
0.08342575
9
0.08223449999999999
19
0.08183075
29
0.08359075
39
0.08167050000000002
49
0.08423200000000003
t:1,predict:1--97676
t:1,predict:0--317037
t:0,predict:1--493504
t:0,predict:0--7171783
0
0.07794274999999999
9
0.0894405
19
0.08040925
29
0.08239775
39
0.0811445
49
0.08514300000000001
t:1,predict:1--98263
t:1,predict:0--316450
t:0,predict:1--491686
t:0,predict:0--7173601
0
0.08224124999999999
9
0.08263300000000001
19
0.08034275
29
0.08283325
39
0.081901
49
0.08275300000000001
t:1,predict:1--97821
t:1,predict:0--316892
t:0,predict:1--491484
t:0,predict:0--7173803
0
0.080348
9
0.08775825000000001
19
0.080602
29
0.0796595
39
0.0833525
49
0.0824095
t:1,predict:1--98363
t:1,predict:0--316350
t:0,predict:1--493005
t:0,predict:0--7172282
0
0.08217750000000001
9
0.08289775
19
0.08581525000000001
29
0.08348749999999999
39
0.07874875
49
0.081605
t:1,predict:1--98263
t:1,predict:0--316450
t:0,predict:1--492893
t:0,predict:0--7172394
0
0.08380425
9
0.08447825
19
0.08383525
29
0.082703
39


29
0.08133099999999999
39
0.07895325
49
0.08330275000000001
t:1,predict:1--98063
t:1,predict:0--316650
t:0,predict:1--492767
t:0,predict:0--7172520
0
0.08160375000000002
9
0.07979125000000001
19
0.0778315
29
0.08335174999999999
39
0.0868095
49
0.08237175
t:1,predict:1--97843
t:1,predict:0--316870
t:0,predict:1--494477
t:0,predict:0--7170810
0
0.08411225000000001
9
0.08346150000000001
19
0.08404
29
0.08094925
39
0.08729525
49
0.08242550000000001
t:1,predict:1--98485
t:1,predict:0--316228
t:0,predict:1--492214
t:0,predict:0--7173073
0
0.08088025000000001
9
0.08199075000000002
19
0.08128575
29
0.07937475
39
0.08338625000000001
49
0.08332275000000001
t:1,predict:1--97759
t:1,predict:0--316954
t:0,predict:1--493770
t:0,predict:0--7171517
0
0.082994
9
0.084686
19
0.08414800000000001
29
0.08193575000000002
39
0.08389799999999999
49
0.084053
t:1,predict:1--98364
t:1,predict:0--316349
t:0,predict:1--493287
t:0,predict:0--7172000
0
0.085741
9
0.08307800000000001
19
0.07901599999999999
29
0.08366

19
0.08099324999999999
29
0.08103775
39
0.08071199999999999
49
0.08465175000000001
t:1,predict:1--97872
t:1,predict:0--316841
t:0,predict:1--494512
t:0,predict:0--7170775
0
0.08162725
9
0.08418075
19
0.07772200000000001
29
0.0863815
39
0.0811665
49
0.0817885
t:1,predict:1--98212
t:1,predict:0--316501
t:0,predict:1--494543
t:0,predict:0--7170744
0
0.08159000000000001
9
0.08339225
19
0.08624675000000001
29
0.08460100000000001
39
0.083529
49
0.08586725
t:1,predict:1--98265
t:1,predict:0--316448
t:0,predict:1--494970
t:0,predict:0--7170317
0
0.08429249999999999
9
0.08398
19
0.08024324999999999
29
0.08471799999999999
39
0.08386600000000001
49
0.08223849999999999
t:1,predict:1--97886
t:1,predict:0--316827
t:0,predict:1--493686
t:0,predict:0--7171601
0
0.0816615
9
0.07840375
19
0.08496275
29
0.079137
39
0.08172575000000001
49
0.08307125
t:1,predict:1--98063
t:1,predict:0--316650
t:0,predict:1--494456
t:0,predict:0--7170831
0
0.08059975
9
0.08335650000000001
19
0.08424175000000002
29
0.0820159

39
0.08759824999999999
49
0.08200300000000002
t:1,predict:1--97921
t:1,predict:0--316792
t:0,predict:1--494460
t:0,predict:0--7170827
0
0.08434050000000001
9
0.083341
19
0.078668
29
0.080775
39
0.07821975
49
0.08201275000000001
t:1,predict:1--98050
t:1,predict:0--316663
t:0,predict:1--493441
t:0,predict:0--7171846
0
0.08243475
9
0.0827365
19
0.080023
29
0.08359299999999999
39
0.08156800000000002
49
0.0821845
t:1,predict:1--98203
t:1,predict:0--316510
t:0,predict:1--492348
t:0,predict:0--7172939
0
0.085696
9
0.08147000000000001
19
0.08093425
29
0.08528500000000001
39
0.0809235
49
0.08535900000000002
t:1,predict:1--97867
t:1,predict:0--316846
t:0,predict:1--495664
t:0,predict:0--7169623
0
0.08130550000000002
9
0.08568
19
0.08249075000000002
29
0.08061450000000002
39
0.08397775
49
0.08350275
t:1,predict:1--97963
t:1,predict:0--316750
t:0,predict:1--492879
t:0,predict:0--7172408
0
0.08478825
9
0.08224175
19
0.08375450000000001
29
0.08145775
39
0.08165675
49
0.0833425
t:1,predict:1--98578
t